In [262]:
import numpy as np
import pandas as pd
import asyncio
import aiohttp
import json
import csv

ModuleNotFoundError: No module named 'aiohttp'

In [ ]:
import requests
import json
import time
from datetime import datetime
from datetime import date
import numpy as np
import pandas as pd

def get_markets(api = "cryptowatch", active = True):
    """get markets available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            if (active):
                exchanges = {item['exchange'] for item in r if item['active'] is True}
            else:
                exchanges = {item['exchange'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
            

    return exchanges

def get_pairs(api = "cryptowatch", active = True):
    """get pairs available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            if (active):
                pairs = {item['pair'] for item in r if item['active'] is True}
            else:
                pairs = {item['pair'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
    return pairs



In [ ]:
print(len(get_pairs()))
print(len(get_pairs(active=False)))
print(len(get_markets()))
print(len(get_markets(active=False)))

In [ ]:
def get_ohlc(market, pair, period = 1440, api = "cryptowatch", start = "", end = "", local_timezone = True):
    """get OHLC prices available in API"""
    period *= 60
    period = str(period)
    params = []
    url = "https://api.cryptowat.ch/markets/" + market + "/" + pair + "/ohlc"
    print(url)
    if (end == ""):
        before = ""
    else:
        before = time.mktime(time.strptime(end, "%d %m %y %H %M"))
    if (start == ""):
        after = start
    else:
        after = time.mktime(time.strptime(start, "%d %m %y %H %M"))
    if (not before or not after):
        prms = [('before', before), ('after', after)]
    else:
        prms = {'before': int(before) , 'after': int(after) }

#    print('params',prms)
    
    if api == "cryptowatch":
        try:
            r = requests.get(url, params=prms).json()['result'][period]
            #ohlc = [item['pair'] for item in r]
            if (local_timezone):            
                a = pd.DataFrame(r, columns = ['date(local)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]] - time.timezone, unit = 's')
                return a
            else:
                a = pd.DataFrame(r, columns = ['date(utc)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]], unit = 's')
                return a               
                

        except:
            print("Error: disfunctional API from Cryptowatch")
            
#columns = ['period','open','high','low','close','volume','neg']    

In [ ]:
#a = get_ohlc("bitfinex", "btcusd", period = 1, start = time.time()-24*60*60, end = time.time())

s1 = "15 07 18 11 40"
e1 = "15 07 18 11 50"
print(time.timezone)
a = get_ohlc("bitfinex", "btcusd", period = 5, start = s1, end = e1)
b = get_ohlc("bitfinex", "btcusd", period = 5)

In [ ]:
def get_daily_price(asset = "btc", api = "coinmetrics", data_type = "price(usd)", start = 0, end = time.time()):
    if (api == "coinmetrics"):
        try:
            url =  "https://coinmetrics.io/api/v1/get_asset_data_for_time_range/"+asset+"/"+data_type
            url = url + "/"+str(int(start)) +"/"+str(int(end))
            r = requests.get(url).json()['result']
            #print(type(r))
            #print(r)
            a = pd.DataFrame(r, columns = ['date(utc)',str(asset)+":"+str(data_type)])
            print(url)
            a[a.columns[0]] = pd.to_datetime(a[a.columns[0]], unit = 's')

            return a
        except:
            print("Error: disfunctional API from Coinmetrics")            
            
    else:
        return 0



In [263]:
a = get_daily_price()
print(time.time())
a.tail()


https://coinmetrics.io/api/v1/get_asset_data_for_time_range/btc/price(usd)/0/1531865482
1531865890.750768


,date(utc),btc:price(usd)
1901,2018-07-12,6396.78
1902,2018-07-13,6235.03
1903,2018-07-14,6247.50
1904,2018-07-15,6272.70
1905,2018-07-16,6357.01


# Terça-Feira

- finalizar get_ohlc() (from datetime to timestamp)

- atualizar get_pairs() colocando um parâmetro active = True para retornar só pares ativos

```Python 
pairs = {item['pair'] for item in r if item['active'] is True} ```

- O mesmo para mercados

